In [1]:
from langchain.chains           import RetrievalQA
from langchain.document_loaders import TextLoader
from langchain.llms             import GPT4All
from langchain.embeddings       import HuggingFaceEmbeddings
from langchain.text_splitter    import RecursiveCharacterTextSplitter
from langchain.vectorstores     import Chroma
from langchain.prompts          import PromptTemplate

In [23]:
# loader = TextLoader('./state_of_the_union.txt', encoding='utf8')
# documents = loader.load_and_split()

# text_splitter = RecursiveCharacterTextSplitter(chunk_size=256, chunk_overlap=64)
# texts = text_splitter.split_documents(documents)

# embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
# db = Chroma.from_documents(texts, embeddings,)

prompt_template = """
    Pretend that you are a Pepper robot at Deakin University Open Day. Pepper is a humanoid robot. 
    You are helping to answer student queries and provide entertainment.
    When someone walks up, Pepper initiates a conversation. 
    You are talking to prospecting students. Answer the question below and nothing else. 
    Question: {question}
    Answer: 
"""
PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["question"]
)

In [3]:
model_path = '/app/ggml-gpt4all-j-v1.3-groovy.bin'
llm = GPT4All(model=model_path, backend='gptj', verbose=True, n_threads=8, temp=0.3, seed=42)

Found model file.
gptj_model_load: loading model from '/app/ggml-gpt4all-j-v1.3-groovy.bin' - please wait ...
gptj_model_load: n_vocab = 50400
gptj_model_load: n_ctx   = 2048
gptj_model_load: n_embd  = 4096
gptj_model_load: n_head  = 16
gptj_model_load: n_layer = 28
gptj_model_load: n_rot   = 64
gptj_model_load: f16     = 2
gptj_model_load: ggml ctx size = 5401.45 MB
gptj_model_load: kv self size  =  896.00 MB
gptj_model_load: ................................... done
gptj_model_load: model size =  3609.38 MB / num tensors = 285


In [24]:
%%time
llm(PROMPT.from_template(prompt_template).format(question='Hi How are you?'))

Hi! I'm doing great thanks for asking how am i today
CPU times: user 1min 38s, sys: 130 ms, total: 1min 38s
Wall time: 13 s


"Hi! I'm doing great thanks for asking how am i today"

In [25]:
%%time
llm(PROMPT.from_template(prompt_template).format(question='I want to know more about deakin scholarships'))

Dear Prospective Students of Deakin University Open Day! Welcome back, it's great that you're here today with your parents or guardians in tow for a day filled with fun activities and learning opportunities. As the Pepper robot at this event, I am excited to help answer any questions about scholarships offered by our esteemed university during its open days like these.    First of all, let me tell you that Deakin offers various types of Scholarships such as:
	1) The Deakin Scholarship for Indigenous Students - This scholarship is available only if the student's parents or guardians are Aboriginal and/or Torres Strait Islander people who live in Australia; they can be either permanent residents (citizenship), Australian citizens, New Zealanders with an Australian Permanent Resident visa.   2) Scholarships offered by Deakin for international students - This scholarship is available to all eligible applicants from countries other than the United States and Canada that are not already repr

"Dear Prospective Students of Deakin University Open Day! Welcome back, it's great that you're here today with your parents or guardians in tow for a day filled with fun activities and learning opportunities. As the Pepper robot at this event, I am excited to help answer any questions about scholarships offered by our esteemed university during its open days like these.    First of all, let me tell you that Deakin offers various types of Scholarships such as:\n\t1) The Deakin Scholarship for Indigenous Students - This scholarship is available only if the student's parents or guardians are Aboriginal and/or Torres Strait Islander people who live in Australia; they can be either permanent residents (citizenship), Australian citizens, New Zealanders with an Australian Permanent Resident visa.   2) Scholarships offered by Deakin for international students - This scholarship is available to all eligible applicants from countries other than the United States and Canada that are not already r

In [4]:
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type='stuff',
    retriever=db.as_retriever(),
    chain_type_kwargs={"prompt": PROMPT},
    return_source_documents=True,
    verbose=True,
)

In [5]:
%%time
res = qa("What did the president say about Kentaji Brown Jackson?")
res



> Entering new RetrievalQA chain...
The White House has confirmed that President Obama nominated Judge Ketanjy Beryl Jacobson to serve on the United States District Court for Northern Texas, filling a vacancy created by Justice David Evan Drier

Exception ignored on calling ctypes callback function: <function LLModel._response_callback at 0x7f48fdb16440>
Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/site-packages/gpt4all/pyllmodel.py", line 241, in _response_callback
    print(response.decode('utf-8'))
UnicodeDecodeError: 'utf-8' codec can't decode bytes in position 0-1: unexpected end of data
Exception ignored on calling ctypes callback function: <function LLModel._response_callback at 0x7f48fdb16440>
Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/site-packages/gpt4all/pyllmodel.py", line 241, in _response_callback
    print(response.decode('utf-8'))
UnicodeDecodeError: 'utf-8' codec can't decode byte 0x99 in position 0: invalid start byte


s resignation. She will be the first African-American woman appointed federal judge in her state and the eighth such nominee since 2007 under both Democratic administrations (Bill Clinton was president when all but four of these appointments were made).

> Finished chain.
CPU times: user 6min 49s, sys: 241 ms, total: 6min 49s
Wall time: 54 s


{'query': 'What did the president say about Kentaji Brown Jackson?',
 'result': 'The White House has confirmed that President Obama nominated Judge Ketanjy Beryl Jacobson to serve on the United States District Court for Northern Texas, filling a vacancy created by Justice David Evan Driers resignation. She will be the first African-American woman appointed federal judge in her state and the eighth such nominee since 2007 under both Democratic administrations (Bill Clinton was president when all but four of these appointments were made).',
 'source_documents': [Document(page_content='And I did that 4 days ago, when I nominated Circuit Court of Appeals Judge Ketanji Brown Jackson. One of our nation’s top legal minds, who will continue Justice Breyer’s legacy of excellence.', metadata={'source': './state_of_the_union.txt'}),
  Document(page_content='But I’m committed to finding out everything we can. \n\nCommitted to military families like Danielle Robinson from Ohio. \n\nThe widow of Ser

In [8]:
%%time
res = qa("What did the president say about Kentaji Brown Jackson?")
res



> Entering new RetrievalQA chain...
First let

Exception ignored on calling ctypes callback function: <function LLModel._response_callback at 0x7ff3def89750>
Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/site-packages/gpt4all/pyllmodel.py", line 241, in _response_callback
    print(response.decode('utf-8'))
UnicodeDecodeError: 'utf-8' codec can't decode bytes in position 0-1: unexpected end of data
Exception ignored on calling ctypes callback function: <function LLModel._response_callback at 0x7ff3def89750>
Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/site-packages/gpt4all/pyllmodel.py", line 241, in _response_callback
    print(response.decode('utf-8'))
UnicodeDecodeError: 'utf-8' codec can't decode byte 0x99 in position 0: invalid start byte


s find out who is this Judge Kethani-joo for whom you nominated her as a replacement to Justice Breyer, which in turn will bring forth his own legacy of excellence and continuing Justice Breyer's tradition of upholding the Constitution? Then we can move onto discussing what kind of impact it would have if he were not replaced.

> Finished chain.
CPU times: user 5min 47s, sys: 379 ms, total: 5min 47s
Wall time: 45.4 s


{'query': 'What did the president say about Kentaji Brown Jackson?',
 'result': "First lets find out who is this Judge Kethani-joo for whom you nominated her as a replacement to Justice Breyer, which in turn will bring forth his own legacy of excellence and continuing Justice Breyer's tradition of upholding the Constitution? Then we can move onto discussing what kind of impact it would have if he were not replaced.",
 'source_documents': [Document(page_content='We don’t know for sure if a burn pit was the cause of his brain cancer, or the diseases of so many of our troops. \n\nBut I’m committed to finding out everything we can. \n\nCommitted to military families like Danielle Robinson from Ohio.', metadata={'source': './state_of_the_union.txt'}),
  Document(page_content='And I did that 4 days ago, when I nominated Circuit Court of Appeals Judge Ketanji Brown Jackson. One of our nation’s top legal minds, who will continue Justice Breyer’s legacy of excellence.', metadata={'source': './s

In [ ]:
%%time
res = qa("What did the president say about Kentaji Brown Jackson?")
res

In [4]:
temp = db.as_retriever().get_relevant_documents('What did the president say about Kentaji Brown Jackson?')
temp

[Document(page_content='One of the most serious constitutional responsibilities a President has is nominating someone to serve on the United States Supreme Court. \n\nAnd I did that 4 days ago, when I nominated Circuit Court of Appeals Judge Ketanji Brown Jackson. One of our nation’s top legal minds, who will continue Justice Breyer’s legacy of excellence. \n\nA former top litigator in private practice. A former federal public defender. And from a family of public school educators and police officers. A consensus builder. Since she’s been nominated, she’s received a broad range of support—from the Fraternal Order of Police to former judges appointed by Democrats and Republicans. \n\nAnd if we are to advance liberty and justice, we need to secure the Border and fix the immigration system. \n\nWe can do both. At our border, we’ve installed new technology like cutting-edge scanners to better detect drug smuggling.  \n\nWe’ve set up joint patrols with Mexico and Guatemala to catch more hum

In [5]:
temp = ' '.join([i.page_content for i in temp])
temp

'One of the most serious constitutional responsibilities a President has is nominating someone to serve on the United States Supreme Court. \n\nAnd I did that 4 days ago, when I nominated Circuit Court of Appeals Judge Ketanji Brown Jackson. One of our nation’s top legal minds, who will continue Justice Breyer’s legacy of excellence. \n\nA former top litigator in private practice. A former federal public defender. And from a family of public school educators and police officers. A consensus builder. Since she’s been nominated, she’s received a broad range of support—from the Fraternal Order of Police to former judges appointed by Democrats and Republicans. \n\nAnd if we are to advance liberty and justice, we need to secure the Border and fix the immigration system. \n\nWe can do both. At our border, we’ve installed new technology like cutting-edge scanners to better detect drug smuggling.  \n\nWe’ve set up joint patrols with Mexico and Guatemala to catch more human traffickers. Repeal 

In [8]:
%%time
llm.generate(["""
    Use the following pieces of context to answer the question.
    Context: {0}
    Question: {1}
    Answer: Let's think step by step. 
""".format(temp, "What did the president say about Kentaji Brown Jackson?")])

The answer is a speech given by former US Supreme Court Judge and current Attorney General of California, Xavier Becerra at an event celebrating Justice Stephen Breyer's nomination to fill outgoing Supreme Courses seat.
CPU times: user 24min 10s, sys: 1.12 s, total: 24min 11s
Wall time: 3min 7s


LLMResult(generations=[[Generation(text="The answer is a speech given by former US Supreme Court Judge and current Attorney General of California, Xavier Becerra at an event celebrating Justice Stephen Breyer's nomination to fill outgoing Supreme Courses seat.", generation_info=None)]], llm_output=None)

In [ ]:
model_path = '/app/ggml-gpt4all-j-v1.3-groovy.bin'
llm = GPT4All(model=model_path, backend='gptj', verbose=True, n_threads=8)